In [1]:
# !pip install --pre toga

In [2]:
from dataclasses import dataclass, field, asdict
from typing import List

import pandas as pd

In [3]:
@dataclass
class Transaction:
    date: str = ""
    account: int = 0
    title: str = ''
    amount: float = 0.0

In [4]:
@dataclass
class GeneralLedger:
    transactions: List[Transaction] = field(default_factory=list)
    list_transactions: List[Transaction] = field(default_factory=list)
    trial_balance_text: List[Transaction] = field(default_factory=list)
        
    def trial_balance(self):
        df = pd.DataFrame([asdict(txn) for txn in self.transactions])
        return df.pivot_table(
            index=['account', 'title'],
            values='amount',
            aggfunc=sum
        ).reset_index()
    
    def add_transaction(self, txn):
        self.transactions.append(txn)
        self.list_transactions = [(str(x.date), str(x.account), str(x.title), str(x.amount)) for x in self.transactions]
        self.trial_balance_text = [(x[0], x[1], str(x[2])) for idx, x in self.trial_balance().iterrows()]
        self.trial_balance_text.extend([('', '', '') for _ in range(50)])
    
    def general_ledger(self):
        df = pd.DataFrame([asdict(txn) for txn in self.transactions])
        return df

In [5]:
import toga
from toga.style.pack import Pack, COLUMN




def build(app):
    
    gl = GeneralLedger()

    
    data = gl.list_transactions

    right_container = toga.Table(headings=['Date', 'Account', 'Title', 'Amount'], 
                                data=data,
                               style=Pack(width=300, padding_left=10, padding_right=10))

    trial_balance = toga.Table(headings=['Account', 'Title', 'Amount'],
                            data=gl.trial_balance_text,
                            style=Pack(width=200))
    
    date = toga.TextInput(initial='12/31/19')
    acct = toga.TextInput(initial='1010')
    title = toga.TextInput(initial='Cash')
    amount = toga.TextInput(initial=500.00)
    
    def add_transaction(widget):
        gl.add_transaction(Transaction(date.value, acct.value, title.value, float(amount.value)))
        right_container.data.insert(0, *gl.list_transactions[-1])
        trial_balance.data = gl.trial_balance_text
        return split
    
    left_content = toga.Box(
        children=[date, 
                  acct, 
                  title, 
                  amount, 
                  toga.Button('Add Transaction',
                            on_press=add_transaction,
                             style=Pack(height=30, padding_bottom=10, padding_left=10)),
                  toga.Box(
        children=[trial_balance
            
        ])
                 ],
        style=Pack(direction=COLUMN, padding_top=10, padding_left=10, padding_right=10, width=120)
    )

    left_container = toga.ScrollContainer(style=Pack(width=220))

    left_container.content = left_content

    split = toga.SplitContainer()

    split.content = [left_container, right_container]


    return split


def main():
    return toga.App('GL Detail', 'Transactions', startup=build)


if __name__ == '__main__':
    main().main_loop()

[Winforms] Not implemented: Table.set_on_select()
[Winforms] Not implemented: Table.set_on_select()
